In [16]:
import requests
import re
from bs4 import BeautifulSoup

url = "https://www.amazon.in/realme-Mirror-Black-128GB-Storage/product-reviews/B08VB7T9YN/"

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
print(soup)

reviews = soup.find_all('span', {'class': 'a-size-base review-text review-text-content'})

for review in reviews:
    print(review.text.strip())


<html>
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<title>Page Not Found</title>
<style type="text/css">
            a:link { font-family: verdana,arial,helvetica,sans-serif; color: #004B91; }
            a:visited { font-family: verdana,arial,helvetica,sans-serif; color: #996633; }
            a:active { font-family: verdana,arial,helvetica,sans-serif; color: #FF9933; }
        </style>
</head>
<body bgcolor="#FFFFFF" style="font-family: verdana,arial,helvetica,sans-serif;" text="#000000">
<!--
            To discuss automated access to Amazon data please contact api-services-support@amazon.com.

For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
        -->
<center>
<a href="/ref=cs_404_logo">
<img alt="Amazon" border="0" src="https://

In [13]:
import requests
import re
from bs4 import BeautifulSoup

def get_reviews(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    

    # Find all the review containers
    review_containers = soup.find_all('div', {'data-hook': 'review'})
    print(review_containers)
    
    # Initialize lists to store positive and negative reviews
    positive_reviews = []
    negative_reviews = []

    # Iterate over each review container
    for container in review_containers:
        # Extract the rating of the review
        rating = container.find('i', {'data-hook': 'review-star-rating'}).text.strip()[0]

        # Extract the text of the review
        text = container.find('span', {'data-hook': 'review-body'}).text.strip()

        # Classify the review as positive or negative based on the rating
        if rating in ['4', '5']:
            positive_reviews.append(text)
        elif rating in ['1', '2']:
            negative_reviews.append(text)

    return positive_reviews, negative_reviews

# Example usage: Get reviews for the first page of results for the "iPhone" search query on Amazon
url = 'https://buy.realme.com/in/goods/611'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
product_links = soup.find_all('a', {'class': 'a-link-normal a-text-normal'})


for link in product_links:
    url = 'https://buy.realme.com' + link['href']
    positive_reviews, negative_reviews = get_reviews(url)

    # Save the positive and negative reviews in separate files
    with open('positive_reviews.txt', 'a', encoding='utf-8') as f:
        f.write('\n'.join(positive_reviews))
        f.write('\n')

    with open('negative_reviews.txt', 'a', encoding='utf-8') as f:
        f.write('\n'.join(negative_reviews))
        f.write('\n')


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Step 1: Preprocessing

def preprocess(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords and stem the remaining words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]

    return ' '.join(words)

In [4]:
# Step 2: Load and preprocess the data

positive_reviews = []
with open("positive_reviews.txt") as f:
    positive_reviews = f.readlines()
positive_reviews = [preprocess(review) for review in positive_reviews]

negative_reviews = []
with open("negative_reviews.txt") as f:
    negative_reviews = f.readlines()
negative_reviews = [preprocess(review) for review in negative_reviews]

FileNotFoundError: [Errno 2] No such file or directory: 'positive_reviews.txt'

In [ ]:
# Step 3: Create a training set and a testing set

training_set = []
training_labels = []

testing_set = []
testing_labels = []

for review in positive_reviews[:500]:
    training_set.append(review)
    training_labels.append('positive')
for review in negative_reviews[:500]:
    training_set.append(review)
    training_labels.append('negative')

for review in positive_reviews[500:600]:
    testing_set.append(review)
    testing_labels.append('positive')
for review in negative_reviews[500:600]:
    testing_set.append(review)
    testing_labels.append('negative')

In [ ]:
# Step 4: Vectorize the text

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
training_vectors = vectorizer.fit_transform(training_set)
testing_vectors = vectorizer.transform(testing_set)

# Step 5: Train and evaluate the model
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(training_vectors, training_labels)
accuracy = clf.score(testing_vectors, testing_labels)
print("Accuracy:", accuracy)
